In [5]:
import numpy as np
import math
#""" Clase que representa la capa de un perceptron sigmoide"""
class Perceptron: 
    def __init__(self):
        self.pesos= [] # """Arreglo donde se guardan los pesos"""
        self.datos=[]  #""" Arreglo donde se guardan los inputs"""
        self.b=0  #"""Este valor representa el bais"""
        self.out=0 #""" Este valor represente el output del sigmoide"""
        self.error=0 #""" El error """
        self.delta=0  #""" El delta"""
        self.lr=0.1 #""" El learning rata"""
        self.bout=0  # """Este valor represente en binario el output, por si se necesita"""
    
   # """ Método para agregar el learning rate"""
    def agregarlr(self,valor):
        self.lr=valor
    #""" Agregar pesos"""
    def agregarpeso(self,peso):
        self.pesos.append(peso)
    #""" Agregar los inputs"""    
    def agregardato(self,dato):
        self.datos.append(dato)
    #""" Agregar bais"""    
    def agregarbais(self,bais):
        self.b=bais
     #"""Agregar erro, este se utiliza si este perectron está en la última capa """"
      #   """ya que aquí el erro es la resta del output con el output esperado """   
    def agregarerror(self,valor):
        self.error=valor
   # """ Calcular el output del sigmoide, ya debe estar ingresados lo datos y los pesos"""
    def run(self):
        ps=np.array(self.pesos)
        dt=np.array(self.datos)
        x= ps*dt
        suma=x.sum()
        exponente = -1*(suma+self.b)
        denominador = 1 + math.exp(exponente)
        self.out=1/denominador
        if self.out>=0.5: #"""Criterio para calcular el output binario"""
            self.bout=1
        else:
            self.bout=0
    #"""Agregar datos"""
    def agregardatos(self,dats):
        self.datos=dats
     #"""Vaciar datos"""   
    def vaciardatos(self):
        self.datos=[]
        
    #"""Calcular error, ingresando los delta de la capa siguiente"""
    def calcularerror(self,deltasnextcapa):
        ps=np.array(self.pesos)
        delt=np.array(deltasnextcapa)
        mult=ps*delt
        self.error=mult.sum()
    #"""Calcular de"""    
    def calculardelta(self):
        self.delta=self.error*(self.out*(1-self.out))
        
    def ajustarpesos(self):
        nuevospesos=[]
        x=0
        for peso in self.pesos:
            nuevospesos.append(peso+(self.lr*self.delta*self.datos[x]))
            x=x+1
        self.pesos=nuevospesos
        self.b=self.b + self.lr*self.delta    

In [2]:
class Capa:
    #"""Clase que representa una capa de red neuronal"""
    def __init__(self):
        self.perceptrones=[] #"""Arreglo con los pereceptrones"""
        self.input=[] #"""Inputs que se le pasaran a  los pereceptrones """
        self.output=[] # """Outputs que regresaran los perceptrones"""
        self.deltas=[]  #"""Deltas de cada uno de los perceptrones """
        self.siguientecapa = None # """Siguiente capa"""
        self.anteriorcapa =None  # """Capa anterior"""
    
    def getnextcapa(self, capa):
        self.siguientecapa = capa
    
    def getpreviouscapa(self,capa):
        self.anteriorcapa = capa
   # """Agregar inputs"""
    def getinput(self,datos):
        self.input=datos
    # Agregar perceptron individual
    def getperceptron(self, perceptron):
        self.perceptrones.append(perceptron)
    # Agregar perceptrones arreglo
    
    def getperceptrones(self, arreglo):
        self.perceptrones=arreglo
    
   #"""Recorrido para calcular output """
    def Feeding(self):
        self.output=[]
        for perc in self.perceptrones: #"""Cada perceptron recibe los datos y su respuestas son cargadas al outputs"""
            perc.agregardatos(self.input)
            perc.run()
            self.output.append(perc.out)
        if self.siguientecapa != None: #"""Si no es la última capa, se pasá mi output como input de la siguiente capa"""
            self.siguientecapa.getinput(self.output)
            self.siguientecapa.Feeding()
    
    #"""Recorrido para ajustar pesos"""
    #"""El argumento de esta función puede ser el expectoutout de la última capa, o los deltas si son otras 
     #  capas """
    def backpropagation(self, expectoutput):
        self.deltas=[]
        if self.siguientecapa == None: #"""Si es última capa"""
            x=0
            for perc in self.perceptrones:
                perc.agregarerror(expectoutput[x]-perc.out)# """Cada pereceptron recibe su error"""
                perc.calculardelta()#"""Cada perceptron calcula su delta"""
                self.deltas.append(perc.delta) #"""Se agregan a los delta de la capaa """
                perc.ajustarpesos()
                x=x+1
            self.anteriorcapa.backpropagation(self.deltas) # """Se le pasa los deltas a la capa anterior"""
        else:
            if self.anteriorcapa == None: # """Si es la primera capa, igual solo que no seguimos con la propagación"""
                for perc in self.perceptrones:
                    perc.calcularerror(expectoutput)
                    perc.calculardelta()
                    self.deltas.append(perc.delta)
                    perc.ajustarpesos()
            else:
                for perc in self.perceptrones: #"""No es ni la primera ni la última"""
                    perc.calcularerror(expectoutput)
                    perc.calculardelta()
                    self.deltas.append(perc.deltas)
                    perc.ajustarpesos()
                self.anteriorcapa.backpropagation(self.deltas)

In [9]:
import random
class Fabrica:
    def __init__(self):
        self.lr=0.1 #learning range con el que se crearn los perceptrones
        self.bais=-1 #bais con el que se crearan los peceptrones
        
    def getlr(self,valor):
        self.lr=valor
        
    def getbais(self,valor):
        self.bais=valor
    #Método que crea un perecpetron con cantidad especifica de pesos   
    def crearPerceptron(self,cantPesos):
        per= Perceptron()
        for x in range(cantPesos):
            per.agregarpeso(random.uniform(-2,2))
        per.agregarlr(self.lr)
        per.agregarbais(self.bais)
        return per
    #Método que crea una capa con una cantidad especifica de perceptrones y pesos.
    def crearCapa(self, cantPer, cantPesos):
        cap= Capa()
        for x in range(cantPer):
            cap.getperceptron(self.crearPerceptron(cantPesos))
        return cap    
    
    

In [ ]:
class Red_Neuronal:
    def __init__(self):
        self.capinicial=None #Capa inicial 
        self.capintemedias=None #Arreglo donde se guardaran las capas intemedias, deben ser ingresadas en orden.
        self.capfinal=None
    #Se agrega la capa incial a la red neuronal, se asume que esta tiene los perceptrones y pesos correctos.
    def getcapinicial(self,capa):
        self.capinicial=capa
    #Agregar capa final.
    def getcapfinal(self, capa):
        self.capfinal= capa
    
    def getcapaintermedia(self.capa):
        self.capintemedias.append(capa)
        
    def enlazar(self):
        if self.capintemedias==None:
            self.capinicial.getnextcapa(self.capfinal)
            self.capfinal.getpreviouscapa(self.capinicial)
        else:
            
    

In [24]:
fabri = Fabrica()
capa1 = fabri.crearCapa(4,2)
capa2 = fabri.crearCapa(1,4)


capa1.getnextcapa(capa2)
capa2.getpreviouscapa(capa1)


for z in range(1500):
    capa1.getinput([0,0])
    capa1.Feeding()
    capa2.backpropagation([0])
    capa1.getinput([0,1])
    capa1.Feeding()
    capa2.backpropagation([1])
    capa1.getinput([1,0])
    capa1.Feeding()
    capa2.backpropagation([1])
    capa1.getinput([1,1])
    capa1.Feeding()
    capa2.backpropagation([0])



In [26]:
capa1.getinput([0,0])
capa1.Feeding()

print(capa2.output)

[0.024417523727853795]
